In [27]:
%load_ext autoreload
%autoreload 2
from torch.utils.data.sampler import Sampler
from torch.utils.data.sampler import RandomSampler
from torch.utils.data import Dataset
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


to get the data run
```
wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
tar -xzvf cifar-10-python.tar.gz
rm cifar-10-python.tar.gz 
```

In [4]:
from cifar_utils import get_CIFAR10_data

In [5]:
data = get_CIFAR10_data(num_training=100, num_validation=200)
for k, v in data.items():
  print('%s: ' % k, v.shape)

X_train:  (100, 3, 32, 32)
y_train:  (100,)
X_val:  (200, 3, 32, 32)
y_val:  (200,)
X_test:  (1000, 3, 32, 32)
y_test:  (1000,)


In [28]:
X_labeled = data['X_train']
y_labeled = pd.get_dummies(data['y_train']).values
X_unlabeled = data['X_val']

In [7]:
from dataset import ArrayDataset, MixupLoader

In [8]:
from torch.utils.data import DataLoader

In [9]:
def augment_fn(x): return x

### Torch Version

In [53]:
from torch import nn
class Flatten(nn.Module):
    def forward(self, x): return x.view(x.size(0), -1)

model = nn.Sequential(
    nn.Conv2d(3, 2, 3, stride=1, padding=1),
    Flatten(),
    nn.Linear(2 * 32 * 32, 10),
)

In [54]:
 ds = ArrayDataset(X_labeled, y_labeled, X_unlabeled)

In [59]:
loader = MixupLoader(ds, batch_size=4)
loader.model = model
for x,y in loader:
    print(x,y)

RuntimeError: The size of tensor a (2) must match the size of tensor b (32) at non-singleton dimension 3

In [58]:
%debug

> /Users/shleifer/mixmatch/dataset.py(67)lin_comb()
     65     return numerator / numerator.sum(dim=1, keepdim=True)
     66 
---> 67 def lin_comb(a, b, frac_a): return (frac_a * a) + (1 - frac_a) * b
     68 
     69 # def randperm(t):

ipdb> beta
*** NameError: name 'beta' is not defined
ipdb> u
> /Users/shleifer/mixmatch/dataset.py(82)mixup_torch()
     80     beta = torch.Tensor(np.random.beta(alpha, alpha, x1.shape[0]))
     81     beta = torch.max(beta, 1-beta)
---> 82     return lin_comb(x1, x2, beta), lin_comb(y1, y2, beta)
     83 
     84 

ipdb> beta
tensor([0.8302, 0.8891])
ipdb> x1.shape
torch.Size([2, 3, 32, 32])
ipdb> x1.float() * beta
*** RuntimeError: The size of tensor a (32) must match the size of tensor b (2) at non-singleton dimension 3
ipdb> x1.float() * beta.unsqueeze(1)
*** RuntimeError: The size of tensor a (32) must match the size of tensor b (2) at non-singleton dimension 2
ipdb> exit


In [49]:
ls

MixMatch.ipynb          cifar-10-python.tar.gz  dataset.py
cifar-10-batches-py/    cifar_utils.py          fetch_cifar.sh*


In [20]:
K,T = loader.K, loader.T
C = lambda arrs: np.concatenate(np.expand_dims(arrs, 0))
X_labeled = C([X for X, y in examples if y != -1])
y = np.array([y for X, y in examples if y != -1])
X_unlabeled = C([X for X, y in examples if y == -1])

xb = loader.augment_fn(X_labeled)
n_labeled = len(X_labeled)
ub = torch.cat([torch.Tensor(loader.augment_fn(X_unlabeled)) for _ in range(K)])  # unlabeled
qb = loader.get_pseudo_labels(ub)
Ux = ub
Uy = torch.cat([qb for _ in range(K)], axis=1)
indices = np.random.shuffle(np.arange(len(xb) + len(Ux)))
Wx = C([Ux, xb], axis=0)[indices]
Wy = C([qb, y], axis=0)[indices]
X, p = mixup(xb, Wx[:n_labeled], y, Wy[:n_labeled], alpha)
U, q = mixup(Ux, Wx[n_labeled:], Uy, Wy[n_labeled:], alpha)
#return X, y, n_labeled

NameError: name 'examples' is not defined

### NPY Version

In [ ]:
def model_const(stuff):
    return np.random.randn(stuff.shape[0], 10)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
y_ohe = pd.get_dummies(y_labeled).values

In [ ]:
augment_fn = lambda x: x

In [ ]:
K = 2
T = .75
alpha = .4
xb = augment_fn(X_labeled[:4])
n_labeled = 4
ub = [augment_fn(X_unlabeled[:4]) for _ in range(K)]  # unlabeled
qb = sharpen(sum(map(model, ub)) / K, T)
C = np.concatenate
Ux = C(ub, axis=0)
Uy = C([qb for _ in range(K)], axis=0)
indices = np.random.permutation(np.arange(len(xb) + len(Ux)))
Wx = C([Ux, xb], axis=0)[indices]
print(qb.shape, y.shape)
Uy = C([qb for _ in range(K)], axis=0) # same guessed labels regardless of aug
#pd.get_dummies(y)
Wy = C([Uy, y_ohe[:4],], axis=0)[indices]

In [ ]:
indices

In [ ]:
Wy.shape

In [ ]:
Wx.shape

In [ ]:

del lin_comb
def lin_comb(a, b, frac_a): 
    try:
        return (frac_a * a) + (1 - frac_a) * b
    except ValueError:
        return (np.tile(frac_a, a.shape) * a) + np.tile(1 - frac_a, a.shape) * b
            
            
            
            
            
            
X, p = mixup(xb, Wx[:n_labeled], y, Wy[:n_labeled], alpha)
U, q = mixup(Ux, Wx[n_labeled:], Uy, Wy[n_labeled:], alpha)

In [ ]:
exit

In [ ]:



def sharpen(x, T):
    numerator = x ** (1 / T)
    return numerator / numerator.sum(axis=1, keepdims=True)

def matmul(arr, scalar):
    return arr * scalar



def mixup(x1, x2, y1, y2, alpha):
    beta = np.random.beta(alpha, alpha, x1.shape[0])
    beta = np.maximum(beta, 1 - beta)
    return lin_comb(x1, x2, beta), lin_comb(y1, y2, beta)

class MixMatchLoss(torch.nn.Module):
    def __init__(self, lambda_u=100):
        super().__init__()
        self.lambda_u = lambda_u
    def forward(self, preds, y, n_labeled):
        labeled_loss = self.cross_entropy(preds[:n_labeled], y[:n_labeled])
        unlabeled_loss = F.mse(preds[n_labeled:], y[n_labeled:])
        return labeled_loss + (self.lambda_u * unlabeled_loss)

In [ ]:
np.tile(.6, (32,32)).shape

In [ ]:


def mixmatch(X_labeled, y, X_unlabeled, model, augment_fn, T=0.5, K=2, alpha=0.75):
    """Generate labeled and unlabeled batches for mixmatch. Helpers are below."""
    xb = augment_fn(X_labeled)
    n_labeled = len(xb)
    ub = [augment_fn(X_unlabeled) for _ in range(K)]  # unlabeled
    qb = sharpen(sum(map(model, ub)) / K, T)
    C = np.concatenate
    Ux = C(ub, axis=0)
    Uy = C([qb for _ in range(K)], axis=0)
    indices = np.random.shuffle(np.arange(len(xb) + len(Ux)))
    Wx = C([Ux, xb], axis=0)[indices]
    Wy = C([qb, y], axis=0)[indices]
    X, p = mixup(xb, Wx[:n_labeled], y, Wy[:n_labeled], alpha)
    U, q = mixup(Ux, Wx[n_labeled:], Uy, Wy[n_labeled:], alpha)
    return C([X, U], axis=1), C([p, q], axis=1), n_labeled

In [ ]:
#resnet18.foa

In [ ]:

class MixMatchLoss(torch.nn.Module):
    def __init__(self, lambda_u=100):
        super().__init__()
        self.lambda_u = lambda_u
        self.xent = torch.nn.CrossEntropyLoss()
        self.mse = torch.nn.MSELoss()

    def forward(self, preds, y, n_labeled):
        labeled_loss = self.xent(preds[:n_labeled], y[:n_labeled])
        unlabeled_loss = self.mse(preds[n_labeled:], y[n_labeled:])
        return labeled_loss + (self.lambda_u * unlabeled_loss)
